In [2]:
import requests
from copy import copy

# Establish baseline
r = requests.get("http://localhost:5000")
ciphertext_baseline = r.cookies["authtoken"]
ciphertext_bytes_baseline = bytearray.fromhex(ciphertext_baseline)

In [17]:
# Determine block size
ciphertext_bytes = copy(ciphertext_bytes_baseline)

def determine_block_size(ciphertext_bytes: bytearray):
    count = 0
    for i in range(len(ciphertext_bytes_baseline)):
        ciphertext_bytes[i] ^= 0x01
        cookie = {"authtoken": ciphertext_bytes.hex()}
        r = requests.get("http://localhost:5000/quote", cookies=cookie)
        
        if (r.content != b'No quote for you!'):
            return count
        
        count += 1

    return -1

block_size = determine_block_size(ciphertext_bytes)
print(f'Block size: {block_size}')

Block size: 16


Note when looking at the ciphertext we use the fact that the last block wont change other blocks since it's the last to find the block size. This means that changes in the cipher here shouldn't result in a change in the rest of the cipher when decoding and therefore it should be able to decode still.

A change in the other parts should change more of the ciphertext resulting in a decode failure.

In [46]:
def oracle(block: bytearray, iv: bytearray, ciphertext_bytes: bytearray):
    block = int.to_bytes(int.from_bytes(block) ^ int.from_bytes(iv), length=block_size)
    start_end = block_count * block_size - block_size * (i_block + 1)
    end_start = block_count * block_size - block_size * i_block

    ciphertext_bytes = ciphertext_bytes[:start_end] + block + ciphertext_bytes[end_start:]

    cookie = {"authtoken": ciphertext_bytes.hex()}
    r = requests.get("http://localhost:5000/quote", cookies=cookie)

    #print(r.content)
        
    if (r.content == b'Padding is incorrect.' or r.content == b'PKCS#7 padding is incorrect.' or r.content == b'No quote for you!'):
        return False
    else:
        return True
        



zero_iv = [0] * block_size

ciphertext_bytes = copy(ciphertext_bytes_baseline)

block_count = len(ciphertext_bytes) // block_size
print(block_count)

block = ciphertext_bytes[-16:]
i_block = 0

flag_found = False

for i_block_byte in range(block_size):
    padding_iv = [i_block_byte ^ b for b in zero_iv]

    for i in range(256):
        padding_iv[-(i_block_byte + 1)] = i

        iv = bytes(padding_iv)

        #print(len(ciphertext_bytes))

        if (oracle(block, iv, ciphertext_bytes)):
            if i_block_byte == 0:
                padding_iv[-2] ^= 1

                if not oracle(block, iv, ciphertext_bytes):
                    continue
            flag_found = True
            print(i)
            break

    if (not flag_found):
        raise Exception("no valid padding byte found")
    else:
        flag_found = False
    zero_iv[-(i_block_byte + 1)] = i ^ i_block_byte


7


Exception: no valid padding byte found